In [44]:
# Import Libraries and dependencies
import pandas as pd
import numpy as np
import warnings
from pandas.tseries.offsets import DateOffset
import yfinance as yf
import keras_tuner as kt
from finta import TA
import hvplot.pandas
import holoviews as hv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
warnings.filterwarnings('ignore')

In [2]:
#Get Nasdaq 100(^NDX) for 1d from yahoo finance for MAX period.
nasdaq100_df = yf.download(tickers='^NDX', period='1y', interval='1h')
display(nasdaq100_df)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2021-01-19 09:30:00-05:00,12920.519531,12929.163086,12869.811523,12889.173828,12889.173828,0
2021-01-19 10:30:00-05:00,12890.312500,12906.209961,12861.724609,12874.675781,12874.675781,80236029
2021-01-19 11:30:00-05:00,12878.788086,12929.115234,12873.158203,12926.179688,12926.179688,60515543
2021-01-19 12:30:00-05:00,12926.759766,12980.352539,12926.759766,12975.586914,12975.586914,60771823
2021-01-19 13:30:00-05:00,12975.661133,13000.093750,12970.091797,12989.964844,12989.964844,59533645
...,...,...,...,...,...,...
2022-01-18 11:30:00-05:00,15301.334961,15369.596680,15271.313477,15301.525391,15301.525391,88961039
2022-01-18 12:30:00-05:00,15300.143555,15300.143555,15238.812500,15253.231445,15253.231445,78080431
2022-01-18 13:30:00-05:00,15252.292969,15296.172852,15246.277344,15272.635742,15272.635742,71625152
2022-01-18 14:30:00-05:00,15272.257812,15338.110352,15236.352539,15258.250977,15258.250977,81582347


In [3]:
# Visualize Close data for nasdaq100_df
close_plt = nasdaq100_df['Close'].hvplot(kind='line', title='Nasdaq 100 Close Prices', xlabel='Date', ylabel='Price ($)', line_color='grey')
close_plt

:Curve   [index]   (Close)

In [4]:
# Set period to be used for trading indicators
period = 20

In [5]:
# Use BBANDS and MFI from TA to get trading indicators for nasdaq100_df
bbands_df = TA.BBANDS(nasdaq100_df, period=period)
mfi_df = pd.DataFrame({'MFI':TA.MFI(nasdaq100_df, period=period)})
obv_df = pd.DataFrame({'OBV':TA.OBV(nasdaq100_df)})
display(bbands_df)
display(mfi_df)

,BB_UPPER,BB_MIDDLE,BB_LOWER
2021-01-19 09:30:00-05:00,NaN,NaN,NaN
2021-01-19 10:30:00-05:00,NaN,NaN,NaN
2021-01-19 11:30:00-05:00,NaN,NaN,NaN
2021-01-19 12:30:00-05:00,NaN,NaN,NaN
2021-01-19 13:30:00-05:00,NaN,NaN,NaN
...,...,...,...
2022-01-18 11:30:00-05:00,15989.151768,15606.745898,15224.340029
2022-01-18 12:30:00-05:00,15955.990979,15573.181836,15190.372693
2022-01-18 13:30:00-05:00,15918.559387,15542.627100,15166.694812
2022-01-18 14:30:00-05:00,15866.037935,15510.352832,15154.667729


,MFI
2021-01-19 09:30:00-05:00,NaN
2021-01-19 10:30:00-05:00,NaN
2021-01-19 11:30:00-05:00,NaN
2021-01-19 12:30:00-05:00,NaN
2021-01-19 13:30:00-05:00,NaN
...,...
2022-01-18 11:30:00-05:00,32.265931
2022-01-18 12:30:00-05:00,31.780799
2022-01-18 13:30:00-05:00,36.623012
2022-01-18 14:30:00-05:00,36.744877


In [6]:
obv_df["OBV_EMA"]=obv_df.ewm(com=20).mean()
pd.options.display.float_format = '{:.6f}'.format
obv_df

,OBV,OBV_EMA
2021-01-19 09:30:00-05:00,NaN,NaN
2021-01-19 10:30:00-05:00,-80236029.000000,-80236029.000000
2021-01-19 11:30:00-05:00,-19720486.000000,-49240263.073171
2021-01-19 12:30:00-05:00,41051337.000000,-17663264.157811
2021-01-19 13:30:00-05:00,100584982.000000,14096168.795627
...,...,...
2022-01-18 11:30:00-05:00,4154109712.000000,4238092734.047827
2022-01-18 12:30:00-05:00,4076029281.000000,4230375426.759835
2022-01-18 13:30:00-05:00,4147654433.000000,4226436331.818891
2022-01-18 14:30:00-05:00,4066072086.000000,4218799939.160849


In [7]:
#Visualize data for bbands_df and mfi_df
bbands_plt = bbands_df.hvplot(kind="line", xlabel='Datetime', ylabel='Price ($)', title='Bolinger Bands for NASDAQ 100')
mfi_plt = mfi_df.hvplot(kind='line', xlabel='Datetime', ylabel='MFI Values', title='Money Flow Index for NASDAQ 100')
obv_plt = obv_df.hvplot(kind='line', xlabel='Datetime', ylabel='OBV Values', title='On Balance Volume for NASDAQ 100')

mfi_sell_line = hv.HLine(80).opts(color='red', line_dash='dashed', line_width=2.0)
mfi_buy_line = hv.HLine(20).opts(color='green', line_dash='dashed', line_width=2.0)
display(bbands_plt*close_plt)
display(mfi_plt*mfi_buy_line*mfi_sell_line)
display(obv_plt)

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [index]   (value)
   .Curve.Close :Curve   [index]   (Close)

:Overlay
   .Curve.I  :Curve   [index]   (MFI)
   .HLine.I  :HLine   [x,y]
   .HLine.II :HLine   [x,y]

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [8]:
# Create a copy of nasdaq100_df and concatenate with bbands_df and mfi_df
trading_signals_df = nasdaq100_df.copy()
trading_signals_df = pd.concat([trading_signals_df, bbands_df, mfi_df, obv_df], axis=1)
display(trading_signals_df)

,Open,High,Low,Close,Adj Close,Volume,BB_UPPER,BB_MIDDLE,BB_LOWER,MFI,OBV,OBV_EMA
2021-01-19 09:30:00-05:00,12920.519531,12929.163086,12869.811523,12889.173828,12889.173828,0,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-19 10:30:00-05:00,12890.312500,12906.209961,12861.724609,12874.675781,12874.675781,80236029,NaN,NaN,NaN,NaN,-80236029.000000,-80236029.000000
2021-01-19 11:30:00-05:00,12878.788086,12929.115234,12873.158203,12926.179688,12926.179688,60515543,NaN,NaN,NaN,NaN,-19720486.000000,-49240263.073171
2021-01-19 12:30:00-05:00,12926.759766,12980.352539,12926.759766,12975.586914,12975.586914,60771823,NaN,NaN,NaN,NaN,41051337.000000,-17663264.157811
2021-01-19 13:30:00-05:00,12975.661133,13000.093750,12970.091797,12989.964844,12989.964844,59533645,NaN,NaN,NaN,NaN,100584982.000000,14096168.795627
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-18 11:30:00-05:00,15301.334961,15369.596680,15271.313477,15301.525391,15301.525391,88961039,15989.151768,15606.745898,15224.340029,32.265931,4154109712.000000,4238092734.047827
2022-01-18 12:30:00-05:00,15300.143555,15300.143555,15238.812500,15253.231445,15253.231445,78080431,15955.990979,15573.181836,15190.372693,31.780799,4076029281.000000,4230375426.759835
2022-01-18 13:30:00-05:00,15252.292969,15296.172852,15246.277344,15272.635742,15272.635742,71625152,15918.559387,15542.627100,15166.694812,36.623012,4147654433.000000,4226436331.818891
2022-01-18 14:30:00-05:00,15272.257812,15338.110352,15236.352539,15258.250977,15258.250977,81582347,15866.037935,15510.352832,15154.667729,36.744877,4066072086.000000,4218799939.160849


In [9]:
# Create a trading algorithm using Bollinger Bands
# Set the Signal column
trading_signals_df["Signal_BB"] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
# trading signal adds one for each buy and doesnt buy more until a sell which resets the trade signal back to 0 and vice verca
for index, row in trading_signals_df.iterrows():
    if (row["Close"] < row["BB_LOWER"]) and (trade_signal < 1):
        trading_signals_df.loc[index, "Signal_BB"] = 1.0
        trade_signal += 1
    elif (row["Close"] > row["BB_UPPER"]) and (trade_signal > 0):
        trading_signals_df.loc[index,"Signal_BB"] = -1.0
        trade_signal = 0

trading_signals_df['Signal_BB'].value_counts()

0.000000     1733
1.000000       18
-1.000000      17
Name: Signal_BB, dtype: int64

In [10]:
# Create a trading algorithm using Money Flow Index
# Set Signal column
trading_signals_df['Signal_MFI'] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the MFI is less than the 20 (Oversold)
# where -1 is when the MFI is greater than 80 (Overbought)
# trading signal adds one for each buy and doesnt buy more until a sell which resets the trade signal back to 0 and vice versa
for index, row in trading_signals_df.iterrows():
    if (row['MFI'] > 80) and (trade_signal > 0):
        trading_signals_df.loc[index, 'Signal_MFI'] = -1
        trade_signal = 0
    elif (row['MFI'] < 20) and (trade_signal < 1):
        trading_signals_df.loc[index, 'Signal_MFI'] = 1
        trade_signal += 1
        
trading_signals_df['Signal_MFI'].value_counts()

0.000000     1760
-1.000000       4
1.000000        4
Name: Signal_MFI, dtype: int64

In [11]:
# Create a trading algorithm using On Balance Volume
# Set Signal column
trading_signals_df['Signal_OBV'] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the MFI is less than the 20 (Oversold)
# where -1 is when the MFI is greater than 80 (Overbought)
# trading signal adds one for each buy and doesnt buy more until a sell which resets the trade signal back to 0 and vice versa
for index, row in trading_signals_df.iterrows():
    if (row['OBV'] > row['OBV_EMA']) and (trade_signal > 0):
        trading_signals_df.loc[index, 'Signal_OBV'] = -1
        trade_signal = 0
    elif (row['OBV'] < row["OBV_EMA"]) and (trade_signal < 1):
        trading_signals_df.loc[index, 'Signal_OBV'] = 1
        trade_signal += 1
        
trading_signals_df['Signal_OBV'].value_counts()

0.000000     1555
1.000000      107
-1.000000     106
Name: Signal_OBV, dtype: int64

In [12]:
def signal_plot(df, signal):
    # Visualize entry position relative to close price
    entry = df[df[signal] == 1.0]["Close"].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price ($)',
        width=1000,
        height=400
    )

    # Visualize exit position relative to close price
    exit = df[df[signal] == -1.0]["Close"].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price ($)',
        width=1000,
        height=400
    )
    
    return entry*exit

display(signal_plot(trading_signals_df, 'Signal_BB')*close_plt.opts(title='Bolinger Bands Trading Strategy')*bbands_plt)

display(signal_plot(trading_signals_df, 'Signal_MFI')*close_plt.opts(title='Money Flow Index Trading Strategy'))

display(signal_plot(trading_signals_df, 'Signal_OBV')*close_plt.opts(title='On Trading Strategy'))

:Overlay
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)
   .Curve.Close      :Curve   [index]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [index]   (value)

:Overlay
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)
   .Curve.Close      :Curve   [index]   (Close)

:Overlay
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)
   .Curve.Close      :Curve   [index]   (Close)

In [13]:
# Set trading signals by using daily returns
trading_signals_df['Actual_Returns'] = trading_signals_df['Adj Close'].pct_change()
trading_signals_df = trading_signals_df.dropna()
# Initialize the new Signal column
trading_signals_df['Signal_RTN'] = 0.0

# Generate Signal to buy stock long[1]
trading_signals_df.loc[(trading_signals_df['Actual_Returns'] >= 0), 'Signal_RTN'] = 1

# Generate Signal to sell stock short[0]
trading_signals_df.loc[(trading_signals_df['Actual_Returns'] < 0), 'Signal_RTN'] = 0

display(trading_signals_df['Signal_RTN'].value_counts())
trading_signals_df.head()

1.000000    937
0.000000    812
Name: Signal_RTN, dtype: int64

,Open,High,Low,Close,Adj Close,Volume,BB_UPPER,BB_MIDDLE,BB_LOWER,MFI,OBV,OBV_EMA,Signal_BB,Signal_MFI,Signal_OBV,Actual_Returns,Signal_RTN
2021-01-21 14:30:00-05:00,13401.761719,13416.301758,13397.538086,13412.180664,13412.180664,68158865,13581.542228,13195.311230,12809.080233,82.816419,699233378.000000,417092901.788214,0.000000,0.000000,0.000000,0.000751,1.000000
2021-01-21 15:30:00-05:00,13411.661133,13433.690430,13400.955078,13401.825195,13401.825195,104681309,13589.258784,13220.943799,12852.628814,84.126379,594552069.000000,430654599.595623,0.000000,0.000000,0.000000,-0.000772,0.000000
2021-01-22 09:30:00-05:00,13354.972656,13402.827148,13338.427734,13357.443359,13357.443359,158857623,13579.570954,13245.082178,12910.593401,79.426403,435694446.000000,431028969.500426,0.000000,0.000000,0.000000,-0.003312,0.000000
2021-01-22 10:30:00-05:00,13356.343750,13380.285156,13336.895508,13358.713867,13358.713867,87538182,13568.737848,13266.708887,12964.679925,73.950044,523232628.000000,437700168.383256,0.000000,0.000000,0.000000,0.000095,1.000000
2021-01-22 11:30:00-05:00,13358.163086,13388.600586,13350.684570,13383.756836,13383.756836,62987529,13560.081687,13287.117383,13014.153078,74.021547,586220157.000000,448186644.523891,0.000000,0.000000,0.000000,0.001875,1.000000


In [14]:
# Find the Strategy Returns for the trading strategy. and visualize comparision of actual vs strategy
def compare_returns(df, signal):
    df['Strategy_Returns'] = df['Actual_Returns'] * df[signal].shift()
    return (1 + df[['Actual_Returns','Strategy_Returns']]).cumprod().hvplot(title=f'{signal} Strategy Returns Vs Actual Returns')

compare_returns(trading_signals_df, 'Signal_RTN')

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [15]:
# Set X and y input for NN
X = trading_signals_df[['BB_UPPER', 'BB_MIDDLE', 'BB_LOWER', 'MFI', 'OBV']].shift().dropna().copy()
y = trading_signals_df['Signal_RTN']

display(X.head())
display(y.head())

,BB_UPPER,BB_MIDDLE,BB_LOWER,MFI,OBV
2021-01-21 15:30:00-05:00,13581.542228,13195.311230,12809.080233,82.816419,699233378.000000
2021-01-22 09:30:00-05:00,13589.258784,13220.943799,12852.628814,84.126379,594552069.000000
2021-01-22 10:30:00-05:00,13579.570954,13245.082178,12910.593401,79.426403,435694446.000000
2021-01-22 11:30:00-05:00,13568.737848,13266.708887,12964.679925,73.950044,523232628.000000
2021-01-22 12:30:00-05:00,13560.081687,13287.117383,13014.153078,74.021547,586220157.000000


2021-01-21 14:30:00-05:00   1.000000
2021-01-21 15:30:00-05:00   0.000000
2021-01-22 09:30:00-05:00   0.000000
2021-01-22 10:30:00-05:00   1.000000
2021-01-22 11:30:00-05:00   1.000000
Name: Signal_RTN, dtype: float64

In [16]:
# Set training start and end dates using DateOffset
# Set offset in days to offset data 60s*60m*24h then multiply by the percentage of data to use
offset = round(((int(round(X.index.max().timestamp())) - int(round(X.index.min().timestamp())))/86400)*.75)
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(days= offset)
print(f'Training Start: {training_begin}, Training End: {training_end}')

Training Start: 2021-01-21 15:30:00-05:00, Training End: 2021-10-20 15:30:00-04:00


In [17]:
# Set X_train, y_train, X_test, y_test
x_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
x_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [18]:
# Scale X_training and X_testing sets using StandardScaler()/MinMaxScaler()
scaler = MinMaxScaler(feature_range=(0,1))
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
x_train_scaled

array([[0.27206737, 0.21683929, 0.18590643, 0.90376743, 0.13489796],
       [0.27469179, 0.22503733, 0.19870525, 0.92105767, 0.10819589],
       [0.27139693, 0.23275749, 0.21574091, 0.85902242, 0.06767456],
       ...,
       [0.9315481 , 0.89596751, 0.86414909, 0.82983174, 0.85385156],
       [0.92935382, 0.89994279, 0.8733512 , 0.82620256, 0.86415315],
       [0.92617563, 0.90372319, 0.88304541, 0.82125022, 0.87538344]])

In [19]:
# Create data for using the last 60 signals worth of trading indicators to predict the signal for the next instance
batch = 60
X_train_ar = []
y_train_ar = []

for i in range(batch, len(x_train_scaled)):
    X_train_ar.append(x_train_scaled[i-batch:i])
    y_train_ar.append(y_train.iloc[i])

In [20]:
# Create data array for testing data
X_test_ar = []
y_test_ar = y_test.iloc[60:]

for i in range(batch, len(x_test_scaled)):
    X_test_ar.append(x_test_scaled[i-batch:i])

In [21]:
# Convert data to numpy arrays and then check shape for LSTM(3 dimensional)
X_train_ar, y_train_ar = np.array(X_train_ar), np.array(y_train_ar)
X_train_ar.shape

(1264, 60, 5)

In [22]:
X_test_ar= np.array(X_test_ar)
X_test_ar.shape

(365, 60, 5)

In [23]:
# Set up the Nueral Network model
nnlstm = Sequential()
a = LSTM(240, activation='tanh',return_sequences=True, input_shape=(X_train_ar.shape[1], X_train_ar.shape[2]))
nnlstm.add(a)
nnlstm.add(LSTM(120, activation='tanh', return_sequences=False))
nnlstm.add(Dense(60))
nnlstm.add(Dense(1, activation='sigmoid'))

In [24]:
# Set up the Nueral Network model
nn = Sequential()
nn.add(Dense(50, activation='relu', input_dim= x_train_scaled.shape[1]))
nn.add(Dense(50, activation='relu'))
nn.add(Dense(25, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

In [25]:
# Compile the Nueral Network model
nnlstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Fit the model with training data
nnlstm.fit(X_train_ar, y_train_ar, batch_size=1, epochs=10)

Epoch 1/10
1264/1264 [==============================] - 91s 70ms/step - loss: 0.6952 - accuracy: 0.5340
Epoch 2/10
1264/1264 [==============================] - 105s 83ms/step - loss: 0.6969 - accuracy: 0.5372
Epoch 3/10
1264/1264 [==============================] - 104s 82ms/step - loss: 0.6910 - accuracy: 0.5427
Epoch 4/10
1264/1264 [==============================] - 102s 81ms/step - loss: 0.6902 - accuracy: 0.5427
Epoch 5/10
1264/1264 [==============================] - 106s 84ms/step - loss: 0.6906 - accuracy: 0.5427
Epoch 6/10
1264/1264 [==============================] - 109s 86ms/step - loss: 0.6904 - accuracy: 0.5427
Epoch 7/10
1264/1264 [==============================] - 104s 82ms/step - loss: 0.6906 - accuracy: 0.5427
Epoch 8/10
1264/1264 [==============================] - 98s 78ms/step - loss: 0.6895 - accuracy: 0.5427
Epoch 9/10
1264/1264 [==============================] - 82s 65ms/step - loss: 0.6907 - accuracy: 0.5427
Epoch 10/10
1264/1264 [==============================] - 7

In [28]:
nn.fit(x_train_scaled, y_train, batch_size=1, epochs=100)

Epoch 1/100
1324/1324 [==============================] - 2s 832us/step - loss: 0.6913 - accuracy: 0.5249
Epoch 2/100
1324/1324 [==============================] - 1s 819us/step - loss: 0.6907 - accuracy: 0.5431
Epoch 3/100
1324/1324 [==============================] - 1s 844us/step - loss: 0.6905 - accuracy: 0.5370
Epoch 4/100
1324/1324 [==============================] - 1s 834us/step - loss: 0.6897 - accuracy: 0.5431
Epoch 5/100
1324/1324 [==============================] - 1s 846us/step - loss: 0.6903 - accuracy: 0.5431
Epoch 6/100
1324/1324 [==============================] - 1s 900us/step - loss: 0.6903 - accuracy: 0.5431
Epoch 7/100
1324/1324 [==============================] - 1s 928us/step - loss: 0.6900 - accuracy: 0.5431
Epoch 8/100
1324/1324 [==============================] - 1s 831us/step - loss: 0.6897 - accuracy: 0.5423
Epoch 9/100
1324/1324 [==============================] - 1s 822us/step - loss: 0.6902 - accuracy: 0.5431
Epoch 10/100
1324/1324 [==============================]

In [29]:
pred_lstm = nnlstm.predict(X_test_ar)

In [30]:
pred_nn = nn.predict(x_test_scaled)

In [31]:
pred_lstm_df = pd.DataFrame({'Actual':y_test_ar, 'Pred_LSTM':np.ravel(pred_lstm)})
pred_lstm_df.head()

,Actual,Pred_LSTM
2021-11-02 12:30:00-04:00,0.000000,0.532198
2021-11-02 13:30:00-04:00,0.000000,0.532198
2021-11-02 14:30:00-04:00,1.000000,0.532198
2021-11-02 15:30:00-04:00,1.000000,0.532198
2021-11-03 09:30:00-04:00,0.000000,0.532198


In [32]:
pred_nn_df = pd.DataFrame({'Actual':y_test, 'Pred_NN':np.ravel(pred_nn)})
pred_nn_df.head()

,Actual,Pred_NN
2021-10-20 15:30:00-04:00,1.000000,0.580092
2021-10-21 09:30:00-04:00,1.000000,0.543654
2021-10-21 10:30:00-04:00,0.000000,0.534460
2021-10-21 11:30:00-04:00,1.000000,0.547263
2021-10-21 12:30:00-04:00,1.000000,0.526781


In [33]:
# Create a new column with a value of 1 for buy or 0 for sell using prediction and rounding.
def signal_transform(df, column_name):
    df.loc[(df[column_name] >= 0.5), 'Pred_R'] = 1
    df.loc[(df[column_name] < 0.5), 'Pred_R'] = 0
    df['Pred_R'].value_counts()

signal_transform(pred_lstm_df, 'Pred_LSTM')
signal_transform(pred_nn_df, 'Pred_NN')

In [34]:
# Visualize Pred_R and Actual
display(pred_lstm_df[['Actual', 'Pred_R']].hvplot(title='Actual Vs LSTM Predicted Signals'))
display(pred_nn_df[['Actual', 'Pred_R']].hvplot(title='Actual Vs NN Predicted Signals'))

:NdOverlay   [Variable]
   :Curve   [index]   (value)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [35]:
# Concatenate Actual_Returns to pred_df
pred_lstm_df= pd.concat([pred_lstm_df, trading_signals_df['Actual_Returns']], axis=1).dropna()
pred_nn_df= pd.concat([pred_nn_df, trading_signals_df['Actual_Returns']], axis=1).dropna()

In [36]:
# Visualize Return using Predicted 
print('LSTM RETURNS')
display(compare_returns(pred_lstm_df, 'Pred_R'))
print('DNN RETURNS')
display(compare_returns(pred_nn_df, 'Pred_R'))

LSTM RETURNS


:NdOverlay   [Variable]
   :Curve   [index]   (value)

DNN RETURNS


:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [45]:
print('LSTM REPORT')
print(classification_report(pred_lstm_df['Actual'] ,pred_lstm_df['Pred_R']))
print('DNN REPORT')
print(classification_report(pred_nn_df['Actual'] ,pred_nn_df['Pred_R']))

LSTM REPORT
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       183
         1.0       0.50      1.00      0.67       182

    accuracy                           0.50       365
   macro avg       0.25      0.50      0.33       365
weighted avg       0.25      0.50      0.33       365

DNN REPORT
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       207
         1.0       0.51      1.00      0.68       218

    accuracy                           0.51       425
   macro avg       0.26      0.50      0.34       425
weighted avg       0.26      0.51      0.35       425



In [61]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), activation=hp.Choice("activation", ["relu", "tanh"]), return_sequences=True, input_shape=(X_train_ar.shape[1], X_train_ar.shape[2])))
    model.add(LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), activation=hp.Choice("activation", ["relu", "tanh"]), return_sequences=False))
    model.add(
        Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32), 
            activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"]))
    )
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model




In [62]:
build_model(kt.HyperParameters())

In [63]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="Resources/",
    project_name="LSTM_tuning",
)

In [64]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}


In [65]:
tuner.search(X_train_ar, y_train_ar, epochs=2, validation_data=(X_test_ar, y_test_ar))

Trial 5 Complete [00h 01m 26s]
val_accuracy: 0.4986301362514496

Best val_accuracy So Far: 0.4999999850988388
Total elapsed time: 00h 06m 46s
INFO:tensorflow:Oracle triggered exit


In [66]:
tuner.results_summary()

Results summary
Results in Resources/LSTM_tuning
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 480
activation: tanh
Score: 0.4999999850988388
Trial summary
Hyperparameters:
units: 320
activation: relu
Score: 0.4986301362514496
Trial summary
Hyperparameters:
units: 224
activation: relu
Score: 0.4986301362514496
Trial summary
Hyperparameters:
units: 448
activation: relu
Score: 0.4986301362514496
Trial summary
Hyperparameters:
units: 384
activation: relu
Score: 0.4986301362514496
